<a href="https://colab.research.google.com/github/sholtodouglas/automated_diffusion/blob/main/3D_Photo_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[CVPR 2020] 3D Photography using Context-aware Layered Depth Inpainting**

[project website](https://shihmengli.github.io/3D-Photo-Inpainting/)

- **Prepare environment**


In [21]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install opencv-python==4.2.0.32
!pip3 install vispy==0.6.4
!pip3 install moviepy==1.0.2
!pip3 install transforms3d==0.3.1
!pip3 install networkx==2.3
!sudo apt install sed

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sed is already the newest version (4.4-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


- **Download script and pretrained model**


In [22]:
%cd /content/
!git clone https://github.com/vt-vl-lab/3d-photo-inpainting
%cd 3d-photo-inpainting
!sh download.sh

/content
fatal: destination path '3d-photo-inpainting' already exists and is not an empty directory.
/content/3d-photo-inpainting
mkdir: cannot create directory ‘checkpoints’: File exists
downloading from filebox ...
--2021-09-02 02:32:40--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/color-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206331633 (197M)
Saving to: ‘color-model.pth’

color-model.pth     100%[===================>] 196.77M  14.5MB/s    in 15s     

2021-09-02 02:32:56 (13.0 MB/s) - ‘color-model.pth’ saved [206331633/206331633]

--2021-09-02 02:32:56--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/depth-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request 

- **Switch off off-screen rendering**

In [23]:
!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

- **Please upload `.jpg` files to `/content/3d-photo-inpainting-master/image/`**
  - You can run this step multiple times to upload multiple `.jpg` files.

In [24]:
%cd image
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
%cd ..

/content/3d-photo-inpainting/image


Saving 0x1.jpg to 0x1.jpg
User uploaded file "0x1.jpg" with length 132146 bytes
/content/3d-photo-inpainting


In [25]:
from PIL import Image
import os 

basewidth = 2100
img = Image.open(os.getcwd() + '/image/0x1.jpg')
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize), Image.ANTIALIAS)
img.save(os.getcwd() + '/image/0x1.jpg')

In [26]:
img = Image.open(os.getcwd() + '/image/0x1.jpg')
img.size

(2100, 2100)

- **Execute the 3D Photo Inpainting**
  - Note: The 3D photo generation process usually takes about 2-3 minutes or more depending on the available computing resources.

In [28]:
!python main.py --config argument.yml

running on device 0
  0% 0/1 [00:00<?, ?it/s]Current Source ==>  0x1
Running depth extraction at 1630550787.7631214
BoostingMonocularDepth/inputs/*.jpg
BoostingMonocularDepth/outputs/*.png
device: cuda
Namespace(Final=True, R0=False, R20=False, colorize_results=False, data_dir='inputs/', depthNet=0, max_res=inf, net_receptive_field_size=None, output_dir='outputs', output_resolution=1, pix2pixsize=1024, savepatchs=0, savewholeest=0)
----------------- Options ---------------
                    Final: True                          	[default: False]
                       R0: False                         
                      R20: False                         
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pix2pix/checkpoints         
         colorize_results: False                         
                crop_size: 672                           
                 data_dir: inputs/       

- **The results are stored in the following directories**
  - Corresponding depth map estimated by [MiDaS](https://github.com/intel-isl/MiDaS.git) 
      - E.g. ```/content/3d-photo-inpainting/depth/moon.npy```
  - Inpainted 3D mesh
      - E.g. ```/content/3d-photo-inpainting/mesh/moon.ply```
  - Rendered videos with zoom-in motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_zoom-in.mp4```
  - Rendered videos with swing motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_swing.mp4```
  - Rendered videos with circle motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_circle.mp4```
  
    ![](https://drive.google.com/uc?id=1xBiZAeGfhsoAsG08N5UM8-f993p0G5HF)

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
!ls

argument.yml		 DOCUMENTATION.md  mesh.py	   README.md
bilateral_filtering.py	 download.sh	   mesh_tools.py   requirements.txt
BoostingMonocularDepth	 image		   MiDaS	   utils.py
boostmonodepth_utils.py  LICENSE	   networks.py	   video
checkpoints		 main.py	   __pycache__
depth			 mesh		   pyproject.toml
